<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                André Quezada &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [40]:
# Paquetes necesarios
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización
import joblib
from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler  # Escalado de datos

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [41]:
# URL del dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"

In [42]:
column_names = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]

# Cargar el dataset directamente desde la URL en tu entorno
data = pd.read_csv(url, names=column_names)

# Codificar variables categóricas con LabelEncoder
label_encoders = {col: LabelEncoder() for col in data.columns}
for col in data.columns:
    data[col] = label_encoders[col].fit_transform(data[col])

# Separar variables predictoras y objetivo
X = data.drop(columns=['class'])
y = data['class']

# Dividir datos en conjunto de entrenamiento y prueba con random_state=84
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=84)

In [43]:
# Escalar solo si hay variables numéricas
numerical_columns = X.select_dtypes(include=["int64", "float64"]).columns

if len(numerical_columns) > 0:
    scaler = StandardScaler()
    X[numerical_columns] = scaler.fit_transform(X[numerical_columns])
    print("\nDatos numéricos escalados.")

# Mostrar la descripción del dataset filtrado
print("\nDescripción del dataset después de la limpieza:")
display(X.describe(include="all"))


Datos numéricos escalados.

Descripción del dataset después de la limpieza:


,buying,maint,doors,persons,lug_boot,safety
count,1728.000000,1728.000000,1.728000e+03,1728.000000,1728.000000,1728.000000
mean,0.000000,0.000000,1.644775e-17,0.000000,0.000000,0.000000
std,1.000289,1.000289,1.000289e+00,1.000289,1.000289,1.000289
min,-1.341641,-1.341641,-1.341641e+00,-1.224745,-1.224745,-1.224745
25%,-0.670820,-0.670820,-6.708204e-01,-1.224745,-1.224745,-1.224745
50%,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,0.670820,0.670820,6.708204e-01,1.224745,1.224745,1.224745
max,1.341641,1.341641,1.341641e+00,1.224745,1.224745,1.224745



### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [44]:
# Crear y entrenar un modelo base de Random Forest
modelo_base = RandomForestClassifier(random_state=84)

# Mostrar los hiperparámetros del modelo
print("Hiperparámetros del modelo:")
modelo_base.get_params()

Hiperparámetros del modelo:


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 84,
 'verbose': 0,
 'warm_start': False}

Explicación de hiperparámetros:
* n_estimators: Número de árboles en el bosque.
* max_depth: Profundidad máxima de cada árbol.
* min_samples_split: Muestras mínimas para dividir un nodo.
* criterion: Función de medición de calidad de división.


Voy a utilizar los hiperparámetros: 'n_estimators', 'max_depth', 'min_samples_split'

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [45]:
# Definir los hiperparámetros a optimizar
parametros = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 3, 5],
    'min_samples_split': [2, 5, 10]
}

# Configurar la validación cruzada con 5 folds y random_state=84
k_fold = KFold(n_splits=5, shuffle=True, random_state=84)

In [46]:
# Aplicar GridSearchCV
modelo = GridSearchCV(RandomForestClassifier(random_state=84), parametros, cv=k_fold, scoring='accuracy', n_jobs=-1)
modelo.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=84, shuffle=True),
             estimator=RandomForestClassifier(random_state=84), n_jobs=-1,
             param_grid={'max_depth': [None, 3, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 20, 30]},
             scoring='accuracy')

In [47]:
# Obtener los mejores parámetros y el mejor score
mejores_parametros = modelo.best_params_
mejor_score = modelo.best_score_

# Mostrar los resultados
print("\nParámetros óptimos:", mejores_parametros)
print("Mejor Score:", mejor_score)


Parámetros óptimos: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 30}
Mejor Score: 0.9587558206456339


El modelo optimizado con GridSearchCV logró una precisión del **95.87%**, indicando un alto desempeño en la clasificación. Los hiperparámetros óptimos encontrados fueron max_depth=None, lo que permite que los árboles crezcan sin restricción de profundidad, min_samples_split=2, permitiendo divisiones en cada nodo con el mínimo de muestras, y n_estimators=30, lo que sugiere que 30 árboles fueron suficientes para lograr una buena generalización sin aumentar el costo computacional.

### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [48]:
# Definir los hiperparámetros con al menos 5 valores cada uno
parametros_random = {
    'n_estimators': [10, 20, 30, 50, 100],
    'max_depth': [None, 3, 5, 10, 20],
    'min_samples_split': [2, 5, 10, 15, 20]
}

In [49]:
# Configurar la validación cruzada con 5 folds
k_fold = KFold(n_splits=5, shuffle=True, random_state=84)

In [50]:
# Configurar RandomizedSearchCV con 25 iteraciones
modelo_random = RandomizedSearchCV(
    RandomForestClassifier(random_state=84),
    parametros_random,
    n_iter=25,
    cv=k_fold,
    scoring='accuracy',
    random_state=84,
    n_jobs=-1
)

In [51]:
# Entrenar el modelo con los datos del dataset
modelo_random.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor score
mejores_parametros_random = modelo_random.best_params_
mejor_score_random = modelo_random.best_score_

# Mostrar los resultados
print("\nParámetros óptimos (RandomizedSearchCV):", mejores_parametros_random)
print("Mejor Score:", mejor_score_random)


Parámetros óptimos (RandomizedSearchCV): {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': None}
Mejor Score: 0.9623737770104117


El modelo optimizado con RandomizedSearchCV logró una precisión del **96.23%**, mostrando una ligera mejora respecto a GridSearchCV. Los hiperparámetros óptimos fueron n_estimators=50, indicando que 50 árboles fueron suficientes para lograr un buen rendimiento, min_samples_split=5, lo que evita divisiones excesivas y ayuda a reducir el sobreajuste, y max_depth=None, permitiendo que los árboles crezcan sin restricción.

### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [52]:
# Definir el modelo con los mejores hiperparámetros encontrados
modelo_final = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=5, random_state=84)

# Reentrenar el modelo con los datos de entrenamiento
modelo_final.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=5, n_estimators=50, random_state=84)

In [53]:
# Evaluar el modelo en los datos de prueba
y_pred = modelo_final.predict(X_test)
score_final = accuracy_score(y_test, y_pred)

# Guardar el modelo entrenado
joblib.dump(modelo_final, "modelo_randomforest.pkl")

# Mostrar el resultado
print("\nScore del modelo final en los datos de prueba:", score_final)
print("Modelo guardado como 'modelo_randomforest.pkl'")



Score del modelo final en los datos de prueba: 0.9566473988439307
Modelo guardado como 'modelo_randomforest.pkl'


### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>